## Deskriptive Betrachtung der Quelle
Dependenzen und Daten laden:

In [1]:
from pathlib import Path
from pandas import DataFrame
import pandas as pd

In [2]:
def read_csv(path: Path) -> DataFrame:
    df = pd.read_csv(path)
    return df

In [3]:
df = read_csv("../data/all.csv")


Ermitteln wie Oft die Orte als Absende Orte vorliegen

In [4]:
from aggregate.letter_map import letter_map, places_set

# alle einzelnen Absendeorte initialisieren
absender_set = set()
for item, val in letter_map.items():
    absender_set.add(val[1])
# Zählen der Absendeorte intitialisieren
absender_count = {}
for i in absender_set:
    absender_count[i] = 0
    
# Orte zählen
for item, val in letter_map.items():
    absender_count[val[1]] += 1
    
absender_count

{'Göhrde': 3,
 'Modena': 1,
 'Torgau': 1,
 'München': 1,
 'Celle': 3,
 'Herrenhausen': 53,
 'Bruchhausen': 1,
 'Oranienburg': 1,
 'Clausthal': 1,
 'Linsburg': 11,
 'Lietzenburg': 8,
 'Wien': 27,
 'Berlin': 62,
 'Hannover': 173,
 'Braunschweig': 8,
 'Zellerfeld': 1,
 'Regensburg': 1,
 'Ebstorf': 6,
 'Karlsbad': 1,
 'Leipzig': 1,
 'Rehburg': 1,
 'Wolfenbüttel': 14,
 'Helmstedt': 1,
 'Ferrara': 1,
 'Wolfenbüttel oder Helmstedt': 1}

testen, ob die Anzahl der Absendeorte mit der Anzahl der Briefe übereinstimmt:

In [5]:

absolute_count = 0
for item, val in absender_count.items():
    absolute_count += val
print(absolute_count, len(letter_map))

382 382


### 1. Zum Address-Verhalten
Leibnitz neigt dazu den Absendeort, wenn überhaupt, unten im Brief einzufügen. 
Sophie von Hannover ist deutlich sorgfältiger und setzt oben meist den Absende Ort ein und schließt mit "An Herrn Leibnitz in {Leibnitzs Aufenthaltsort}"

Auslesen, wie hoch der Impact von Absendeort auf Ortsnennung ist:

In [6]:
# Dataframe mit Absendeorten:
absendeorte = df[df['Place'].isin(absender_set)]
ab_all = set(absendeorte['Place'])

missing_places = absender_set - ab_all
print( missing_places)

{'Lietzenburg', 'Rehburg', 'Wolfenbüttel oder Helmstedt'}


## 2. Reise Verhalten

TODO: Karte machen?

In [7]:
for v in missing_places:
    print(v, v in places_set)

Lietzenburg True
Rehburg True
Wolfenbüttel oder Helmstedt False


## 3. Generelles Briefverhalten
Buchstaben gesamt:
* Sophie: 315884
* Leibnitz: 908847

Wörter gesamt: 
* Sophie: 45842
* Leibnitz: 125487

In [8]:
from aggregate.letter_map import letter_map
#read in data
import pandas as pd
places_sophie = pd.read_csv("../data/per person/Sophie.csv")
places_leibnitz = pd.read_csv("../data/per person/Leibnitz.csv")

In [9]:
# descriptive analysis about letters
from pathlib import Path

total_characters_sophie = 0
total_characters_leibnitz = 0
total_words_sophie = 0
total_words_leibnitz = 0

paths = [path for path in Path("../letters/cleaned").iterdir() if path.is_file()]

for path in paths:
    letter_num = int(path.stem.split("_")[-1])
    active_person = letter_map[letter_num][0]
    with open(path, "r") as f:
        body = f.read()
        num_chars = len(body)
        num_words = len(body.split(" "))
        if active_person == "Sophie":
            total_characters_sophie += num_chars
            total_words_sophie += num_words
        elif active_person == "Leibnitz":
            total_characters_leibnitz += num_chars
            total_words_leibnitz += num_words
        else:
            print(f"found rouge letter: {letter_num}")
        

In [10]:
print(total_characters_sophie, total_words_sophie)
print(total_characters_leibnitz, total_words_leibnitz)

315884 45842
908847 125487


### 3.1 Anzahl Briefe
* Sophie: 171
* Leibnitz: 211

In [11]:
# amount of letters per person
## Sophie
num_letters_sophie = 0
## Leibtnitz
num_letters_leibnitz = 0

for item, val in letter_map.items():
    if val[0] == "Sophie":
        num_letters_sophie += 1
    elif val[0] == "Leibnitz":
        num_letters_leibnitz +=1
    else:
        print(f"rouge letter found: {item}")

print(num_letters_sophie, num_letters_leibnitz)        

171 211


In [12]:
# characters/ words per letter:
characters_per_letter_sophie = total_characters_sophie/num_letters_sophie
words_per_letter_sophie = total_words_sophie / num_letters_sophie

characters_per_letter_leibnitz = total_characters_leibnitz/num_letters_leibnitz
words_per_letter_leibnitz = total_words_leibnitz / num_letters_leibnitz

print(f"sophie: characters per letter: {characters_per_letter_sophie} | words per letter: {words_per_letter_sophie}")
print(f"leibnitz: characters per letter: {characters_per_letter_leibnitz} | words per letter: {words_per_letter_leibnitz}")



sophie: characters per letter: 1847.2748538011697 | words per letter: 268.08187134502924
leibnitz: characters per letter: 4307.331753554502 | words per letter: 594.7251184834123


sophie: characters per letter: 1847.2748538011697 | words per letter: 268.08187134502924

leibnitz: characters per letter: 4307.331753554502 | words per letter: 594.7251184834123

### 3.2 Orts Dichte in Briefen

insgesamt:
* Sophie: 1178
* Leibnitz: 2231

pro brief:
* Sophie: 6.888888888888889
* Leibnitz: 10.57345971563981

Verhältnis von Ort pro Wort in Brief:
* Sophie: 38.9151103565365
* Leibnitz: 56.24697445091887
 

In [13]:
# total number of letters per person
total_letters_sophie = sum(places_sophie['Count'])
total_letters_leibnitz = sum(places_leibnitz['Count'])
print(total_letters_sophie, total_letters_leibnitz)

1178 2231


In [14]:
# place density per letter per person
density_sophie = total_letters_sophie / num_letters_sophie
density_leibnitz = total_letters_leibnitz / num_letters_leibnitz
print(density_sophie, density_leibnitz)

6.888888888888889 10.57345971563981


In [15]:
# place per word per letter 
place_per_word_sophie =  words_per_letter_sophie / density_sophie
place_per_word_leibnitz = words_per_letter_leibnitz / density_leibnitz

print(place_per_word_sophie, place_per_word_leibnitz)

38.9151103565365 56.24697445091887


### 3.3 Vergleich ob person Orte öfter nennt (normalisiert/nicht normalisiert)

In [16]:
import pandas as pd

sophie = pd.read_csv("../data/per person/Sophie.csv")
leibnitz = pd.read_csv("../data/per person/Leibnitz.csv")

In [17]:
sophie


,Place,Count
0,Aachen,2
1,Adda,1
2,Afrika,1
3,Amsterdam,2
4,Anhalt,5
...,...,...
194,Wolfenbüttel,22
195,Württemberg,3
196,Zell,1
197,Zweibrücken,3


In [18]:
leibnitz

,Place,Count
0,Afrika,2
1,Ahlden,2
2,Alpen,1
3,Alster,1
4,Altdorf,3
...,...,...
353,Ziesar,1
354,Ägypten,2
355,Ägypter,3
356,Äthiopien,1


In [19]:
all = pd.read_csv("../data/all.csv")

In [20]:
all

,Place,Count
0,Aachen,2
1,Adda,1
2,Afrika,3
3,Ahlden,2
4,Alpen,1
...,...,...
407,Zweibrücken,3
408,Ägypten,3
409,Ägypter,3
410,Äthiopien,1


In [21]:
from aggregate.letter_map import nationalities_ethnicities
et = all[all['Place'].isin(nationalities_ethnicities)]

In [22]:
et

,Place,Count
15,Armenier,1
19,Australier,1
88,Engländer,40
94,Europäer,1
101,Franzose,2
141,Holländer,15
154,Italiener,6
193,Litauer,1
283,Pole,1
288,Portugiesen,1


## Testen ob das Inhaltsverzeichnis stimmt?

In [23]:
from aggregate.letter_map import letter_map

In [24]:
s1 = """747 365. Kurfürstin Sophie an Leibniz, Herrenhausen, 1. August 1713
748 366. Leibniz an Kurfürstin Sophie, Wien, 29. November 1713
752 367. Kurfürstin Sophie an Leibniz, Hannover, 8. Dezember 1713
753 368. Kurfürstin Sophie an Leibniz, Hannover, 28. Dezember 1713
755 369. Leibniz an Kurfürstin Sophie, Wien, 1713
757 370. Kurfürstin Sophie an Leibniz, Hannover, 4. Januar 1714
758 371. Kurfürstin Sophie an Leibniz, Hannover, Ende Januar 1714
761 372. Leibniz an Kurfürstin Sophie, Wien, 31. Januar 1714
765 373. Kurfürstin Sophie an Leibniz, Hannover, 13. Februar 1714
767 374. Kurfürstin Sophie an Leibniz, Hannover, 1. März 1714
769 375. Kurfürstin Sophie an Leibniz, Hannover, 1. März 1714
771 376. Kurfürstin Sophie an Leibniz, Hannover, 7. März 1714
772 377. Kurfürstin Sophie an Leibniz, Hannover, 2. April 1714
775 378. Leibniz an Kurfürstin Sophie, Wien, 7. April 1714
779 379. Leibniz an Kurfürstin Sophie, Wien, 9. Mai 1714
783 380. Kurfürstin Sophie an Leibniz, Hannover, 20. Mai 1714
785 381. Leibniz an Kurfürstin Sophie, Wien, 24. Mai 1714
787 382. Leibniz auf Kurfürstin Sophie, Wien, Ende Juni oder Anfang Juli 1714 (?)"""

In [25]:
person_map = {"Herzogin": "Sophie","Kurfürstin":"Sophie", "Leibniz": "Leibnitz"}

for line in s1.split("\n"):
    split = line.split()
    letter_num = int(split[1].split(".")[0])
    pred_person = letter_map[letter_num][0] 
    true_person = split[2]
    if pred_person != person_map[true_person]:
        print("wrong!", letter_num)
    # else:
    #     print(pred_person == person_map[true_person])

clean of nationalitites etc.

In [26]:
from aggregate.letter_map import nationalities_ethnicities, rivers

In [27]:
all = all.sort_values(by=['Count'], ascending=False)

In [28]:
all=all[~all["Place"].isin(nationalities_ethnicities)]

In [29]:
all[all["Place"].isin(rivers)]

,Place,Count
304,Rhein,13
77,Donau,8
85,Elbe,5
205,Maas,4
310,Ruhr,4
357,Stör,2
282,Po,2
149,Inn,2
84,Eider,1
91,Etsch,1


In [30]:
all.head(20)

,Place,Count
128,Hannover,265
30,Berlin,178
87,England,158
40,Braunschweig,142
100,Frankreich,127
331,Schweden,122
396,Wien,110
293,Preußen,98
284,Polen,92
60,Celle,75


In [31]:
all

,Place,Count
128,Hannover,265
30,Berlin,178
87,England,158
40,Braunschweig,142
100,Frankreich,127
...,...,...
137,Hessen-Kassel,1
135,Herzberg,1
294,Pruth,1
133,Herford,1
